In [1]:
%matplotlib inline

from astropy.io import fits
import healpy as hp
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.special import gamma as gafun
from scipy.stats import norm
import scipy.integrate as integrate
import os
import time
import shelve
import glob
import warnings

import pandas
from astropy.table import Table
import numpy.ma as ma


try:
    import camb
    usecamb=True
except ModuleNotFoundError:
    print('Module "camb" not found, please input the Cl you want to use')
    usecamb=False

In [2]:
nside=2048
npix=12*nside**2
B=1.2
jlist=np.array([38,39,40])
folder='/home/javier/data/Thesis/run4'
mapsfolder='/home/javier/data/Thesis/maps/'
masksfolder='/home/javier/data/Thesis/masks/'
catalogsfolder='/home/javier/data/Thesis/catalogs/'
imap=0
alphalist=np.array([0.05,0.01,0.002,0.])
angtol=3./60.
shelvefile=folder+'/shelve'

try:
    my_shelve = shelve.open(folder+'/shelve')
    results=my_shelve['results']
    catresults=my_shelve['catresults']
    totresults=my_shelve['totresults']
    my_shelve.close()
    imap=len(results.keys())
except KeyError:
    results=pandas.DataFrame()

In [349]:
def loadmap(file,colname=None,ext=1,nside=nside):
    hdu=fits.open(file)
    if colname==None:
        return(print('The available colnames are \n'+str(hdu[ext].columns)))
    else:
        mapp=hdu[ext].data[colname]
    mapp=mapp.flatten()
    if hdu[ext].header['ORDERING']=='NESTED':
        mapp=hp.reorder(mapp,n2r=True)
    if hdu[ext].header['nside']!=nside:
        mapp=hp.ud_grade(mapp,nside_out=nside)
    hdu.close()
    return(mapp)

def needlet(B,j,lmax):
    ls=np.arange(lmax+1)
    u=(ls/B**j)
    return(u**2.*np.exp(-u**2.))


def getexactks(clsj,ls,maxl=None):
    if maxl is not None:
        ls=ls[:maxl]
        clsj=clsj[:maxl]
    if len(clsj) != len(ls):
        raise ValueError('clsj and ls must have the same lenght')
    c1=np.sum((2.*ls+1.)*ls*(ls+1.)/(8.*np.pi)*clsj)
    c2=np.sum((2.*ls+1.)*ls*(ls+1.)*(ls-1.)*(ls+2.)/(32.*np.pi)*clsj)
    k1=c1/c2
    k2=c1**2./c2
    return(k1,k2)

def getcs(gamma,n,p=1.):
    '''
    Outputs c_{p,2n}(gamma)
    '''
    return(2.**(gamma/2. - 2. - n - 2.*p) * gafun(1.- gamma/2. + n +2.*p))
    
def getks(j,B,gamma,p=1.):
    #Approximation as j goes to infinity
    cp0=getcs(gamma,0.,p)
    cp2=getcs(gamma,1.,p)
    cp4=getcs(gamma,2.,p)
    k1=4.*(cp2/cp4)*(B**(-2.*j))
    k2=2.*(cp2)**2./(cp0*cp4)
    return(k1,k2)
    
def funf(x,k1,k2):
    fx=((2.*np.sqrt(3.+k1))/(2.+k1*np.sqrt(3.+k1))* 
    ((k1+k2*(x**2.-1.))*norm.pdf(x)*norm.cdf((x*np.sqrt(k2))/(np.sqrt(2+k1-k2))) +
    np.sqrt(k2*(2.+k1-k2))/(2.*np.pi)*x*np.exp((-(2.+k1)*x**2.)/(2.*(2.+k1-k2))) +
    np.sqrt(2./(np.pi*(3.+k1-k2)))*np.exp((-(3.+k1)*x**2.)/(2.*(3.+k1-k2)))*
    norm.cdf((np.sqrt(k2)*x)/np.sqrt((2.+k1-k2)*(3.+k1-k2)))
    ))
    return(fx)

def pval(x,k1,k2):
    if x<37.5:
        result=integrate.quad(funf,x,np.inf, args=(k1,k2))
    else:
        result=np.array([0,0])
    return(result)

def percent(perc, cbins, num):
    nbins=len(cbins)
    minmax=np.zeros((2,nbins))
    snum=np.sort(num,axis=0).T
    for ibin in np.arange(nbins):
        minmax[:,ibin]=np.interp([1.-perc,perc],(np.arange(nmaps)+1)/nmaps,snum[ibin])
    return(minmax)

def blf(l,j,B=1.2):
    u=(l/B**j)
    bl=u**2.*np.exp(-u**2.)
    return(bl)

vfunf=np.vectorize(funf)
vpval=np.vectorize(pval)

class Catalogs(object):
    def __init__(self,folder,key='',notkey='zzzzzzz'):
        self.folder=folder
        files=[s for s in os.listdir(folder) if key in s and notkey not in s]
        self.catlist={}
        for file in files:
            hdu=fits.open(folder+file)
            self.catlist[file]=Table(hdu[1].data).to_pandas()
            hdu.close()
        
    def check(self,angs,tol=1):
        dets=0
        if angs.shape == (2,):
            angs=np.array(angs)
        for [lon,lat] in angs:
            for tab in self.catlist:
                det=np.sum(((self.catlist[tab]['GLON']-lon)**2. + (self.catlist[tab]['GLAT']-lat)**2.).between(-1,tol**2.))
                if det>0:
                    dets=dets+1
                    break
        return(dets)
        
def show_max(mapp,pix):
    pix=int(pix)
    ang=hp.pix2ang(nside,pix,lonlat=True)
    plt.figure(figsize=(8,8))
    ax=plt.gca()
    hp.gnomview(mapp,rot=(ang[0],ang[1],0.), hold=True, title=f'CMB map, S/N={mapp[pix]/np.std(mapp)}')
    return(ax)

def show_maxb(mapp,betas,pix):
    pix=int(pix)
    ang=hp.pix2ang(nside,pix,lonlat=True)
    fig,([ax1,ax2])=plt.subplots(ncols=2,nrows=1,figsize=(8,8))
    plt.sca(ax1)
    hp.gnomview(mapp,rot=(ang[0],ang[1],0.), hold=True, title=f'CMB map, S/N={mapp[pix]/np.std(mapp)}')
    plt.sca(ax2)
    hp.gnomview(betas,rot=(ang[0],ang[1],0.), hold=True, title=f'Beta map, S/N={betas[pix]/np.std(betas)}')
    return(fig,([ax1,ax2]))

def find_map(num,results):
    if results.T[num]["inpainted"]:
        colname='I_STOKES_INP'
    else:
        colname='I_STOKES'
    mapp=loadmap(f'/home/javier/data/Thesis/maps/COM_CMB_IQU-{results.T[num]["Map"]}_2048_R{results.T[num]["version"]}_full.fits',colname=colname)
    return(mapp)

def locate(mapp,angs):
    hp.mollview(mapp)
    hp.projscatter((angs).T,lonlat=True,color='r',facecolor='none')
    
def locate_row(num,results):
    mapp=find_map(num,results)
    angs=results.T[num]["angs"]
    locate(mapp,angs)

In [123]:
# mask40=loadmap(maskfile,colname='GAL040',nside=nside)
# mask70=loadmap(maskfile,colname='GAL070',nside=nside)
# mask90=loadmap(maskfile,colname='GAL090',nside=nside)
# mask97=loadmap(maskfile,colname='GAL097',nside=nside)


NameError: name 'maskfile' is not defined

In [42]:
l=np.arange(3*nside)
suma=np.zeros(np.shape(l))
for j in np.arange(500):
    suma=suma+blf(l=l,B=1.2,j=j)**2.
K=suma[3:].mean()

In [4]:
maskinp=loadmap(masksfolder+'COM_Mask_CMB-Inpainting-Mask-Int_2048_R3.00.fits',colname='TMASK',nside=nside)

In [5]:
maskcom=loadmap(masksfolder+'COM_Mask_CMB-common-Mask-Int_2048_R3.00.fits',colname='TMASK',nside=nside)

In [207]:
time1=time.time()
my_shelve = shelve.open(shelvefile,'n')      #'n' for new

for file in os.listdir(mapsfolder):
    version=file[-14:-10]
    namedict={'r':'commander','c':'nilc','m':'sevem','a':'smica'}
    mapname=namedict[file[-22]]
    if version=='2.01':
        colnames=['I_STOKES']
    if version=='3.00':
        colnames=['I_STOKES','I_STOKES_INP']
        
    for colname in colnames:
        inpaint=(colname=='I_STOKES_INP')

        mapp=loadmap(mapsfolder+file,colname=colname,nside=nside)

        alm=hp.map2alm(mapp)

#         cls=hp.alm2cl(alm)
#         ls=np.arange(len(cls))

        for j in jlist:

            betas=hp.alm2map(hp.almxfl(alm,needlet(B,j,3*nside)),nside)/np.sqrt(npix);

            cmbsigma=np.std(ma.masked_array(mapp, mask=(1-maskcom)))
            betasigma=np.std(ma.masked_array(betas, mask=(1-maskcom)))
            
            betas[betas>10000*betasigma]=9999.*betasigma

            hp.write_map(folder+'/beta'+f'{imap}'+'.fits',betas/betasigma,column_names=['BETA'],column_units=['uK'],overwrite=True);

            hotspotfile= open(folder+f'/hotspot{imap}.par','w')
            hotspotfile.writelines(['infile='+folder+'/beta'+str(imap)+'.fits \n',
            'extrema_outfile = '+folder+'/pixlminmax'+str(imap)+'.fits \n',
            'maxima_outfile = '+folder+'/maxima'+str(imap)+'.dat \n',
            'minima_outfile = '+folder+'/minima'+str(imap)+'.dat'])
            hotspotfile.close()

            !hotspot {folder}/hotspot{imap}.par > /dev/null

            maxi=np.loadtxt(folder+'/maxima'+str(imap)+'.dat')
            maxs=np.array(maxi)[:,1]

            ######### To check the cls of the maps
            # clsj=hp.sphtfunc.anafast(betas)

            # print(np.std(betas),betasigma)

            # plt.figure()
            # plt.plot(ls,cls/betasigma**2.*needlet(B=B,j=j,lmax=3*nside-1)**2./(cmbsigma**2.*npix**2.)*K) #number from the normalization of the needlet
            # plt.plot(ls,clsj/betasigma**2.)
            #########

            ######### To plot the maxima distribution and the fits
            # plt.figure()
            # num,bins,patches=plt.hist(maxs,bins=40,density=1,range=(-5,10))
            # cbins=(bins[:-1]+bins[1:])/2

            k1apro,k2apro=getks(gamma=2.5,B=B,j=j)
            # theo1=vfunf(cbins,k1apro,k2apro)

            # k1theo,k2theo=getexactks(cls/betasigma**2.*needlet(B=B,j=j,lmax=3*nside-1)**2./(cmbsigma**2.*npix**2.)*K**2.,ls)
            # theo2=vfunf(cbins,k1theo,k2theo)

            # print(k1apro,k2apro)
            # print(k1theo,k2theo)
            # print(getexactks(clsj/betasigma**2.,ls))

            # plt.figure()
            # plt.plot(cbins,num, 'k', label='Mean')
            # plt.plot(cbins,theo1, 'r', label='Analytical aprox')
            # plt.plot(cbins,theo2, 'c--', label='Analytical exact')
            # plt.gca().set_title(f'j={j}')
            # plt.legend()

            # plt.figure()
            # plt.plot(cbins,num-theo1, 'k', label='Difference')
            # plt.gca().set_title(f'j={j}')
            #########



            sortmax=maxi[maxi[:,1].argsort()]
            step=0.05
            xp=np.arange(sortmax[0,1]-step,sortmax[-1,1]+step,step)

            fp=vpval(xp,k1=k1apro,k2=k2apro)
            apvalues=np.interp(sortmax[:,1], xp, fp[0])
            spv=np.sort(apvalues)

            for alpha in alphalist:
                vec=np.arange(1,len(spv)+1)*alpha

#                 show=False
#                 maxcandidates=5
                if np.shape(np.argwhere(vec>=spv*len(sortmax[:,1])))[0] > 0:
                    index=np.argwhere(vec>=spv*len(sortmax[:,1]))[-1][0]
#                     if show==True:
#                         print('The first '+str(index+1)+' maxima are likely to be point sources.')
#                         plt.figure()
#                         plt.plot(spv[:2*index+2]*len(sortmax[:,1]))
#                         plt.plot(vec[:2*index+2])
                    canlist=sortmax[-index-1:]
                    angs=np.array(hp.pix2ang(nside,canlist[:,0].astype('int'),lonlat=True)).T

#                     if show==True:
#                         for [pix,inten] in canlist[-maxcandidates:]:
#                             show_max(mapp,betas,pix)

                else:
                    index=0
#                     if show==True:
#                         print('No maxima is expected to be a point source')
#                         plt.figure()
#                         plt.plot(spv[:100]*len(sortmax[:,1]))
#                         plt.plot(vec[:100])
                    canlist=np.array([])
                    angs=np.array([])

                
                index=['Map','version','inpainted','B','j','nside','alpha','detections','outside common mask','outside inpainting mask','canlist','angs']
                if len(angs)>0:
#                     inmask40=np.sum(1-np.isin(canlist[:,0],np.where(mask40==0)))
#                     inmask70=np.sum(1-np.isin(canlist[:,0],np.where(mask70==0)))
#                     inmask90=np.sum(1-np.isin(canlist[:,0],np.where(mask90==0)))
                    inmaskcom=np.sum(1-np.isin(canlist[:,0],np.where(maskcom==0)))
                    inmaskinp=np.sum(1-np.isin(canlist[:,0],np.where(maskinp==0)))

                else:
#                     inmask40=0
#                     inmask70=0
#                     inmask90=0
                    inmaskcom=0
                    inmaskinp=0

                data=[mapname,version,inpaint,B,j,nside,alpha,len(angs),inmaskcom,inmaskinp,canlist,angs]

                results[imap]=pandas.Series(data,index=index)
                imap+=1

            print('Map number '+str(imap+1))
    display(results.T)
    my_shelve['results'] = results
    
my_shelve.close()

time2=time.time()
time.strftime("%H:%M:%S", time.gmtime(time2-time1))

Map number 5
Map number 9
Map number 13


,Map,version,inpainted,B,j,nside,alpha,detections,outside common mask,outside inpainting mask,canlist,angs
0,smica,2.01,False,1.2,38,2048,0.05,365,14,25,"[[17969506.0, 4.725307], [42997244.0, 4.734639...","[[15.57861328125, 16.60711032435087], [182.185..."
1,smica,2.01,False,1.2,38,2048,0.01,294,1,2,"[[25292343.0, 5.105129], [25129117.0, 5.109422...","[[339.9169921875, -0.2797656601000824], [6.921..."
2,smica,2.01,False,1.2,38,2048,0.002,242,0,0,"[[25276177.0, 5.470043], [25120330.0, 5.470631...","[[349.4970703125, -0.24246333228779804], [340...."
3,smica,2.01,False,1.2,38,2048,0,0,0,0,[],[]
4,smica,2.01,False,1.2,39,2048,0.05,728,22,54,"[[24016663.0, 4.629977], [25776567.0, 4.630096...","[[79.78271484375, 2.612040471496698], [19.2919..."
5,smica,2.01,False,1.2,39,2048,0.01,558,2,10,"[[41960069.0, 5.042193], [25103369.0, 5.042642...","[[208.74633431085044, -41.86035581487161], [31..."
6,smica,2.01,False,1.2,39,2048,0.002,482,0,2,"[[24344654.0, 5.388754], [25498200.0, 5.392681...","[[93.44970703125, 1.865426534502518], [26.3671..."
7,smica,2.01,False,1.2,39,2048,0,0,0,0,[],[]
8,smica,2.01,False,1.2,40,2048,0.05,1565,39,180,"[[24123141.0, 4.542908], [10469814.0, 4.543569...","[[78.9697265625, 2.3693484086689125], [199.270..."
9,smica,2.01,False,1.2,40,2048,0.01,1197,4,63,"[[25554534.0, 4.956656], [24285589.0, 4.959849...","[[341.982421875, -0.8766297866493602], [17.797..."


Map number 17
Map number 21
Map number 25


,Map,version,inpainted,B,j,nside,alpha,detections,outside common mask,outside inpainting mask,canlist,angs
0,smica,2.01,False,1.2,38,2048,0.05,365,14,25,"[[17969506.0, 4.725307], [42997244.0, 4.734639...","[[15.57861328125, 16.60711032435087], [182.185..."
1,smica,2.01,False,1.2,38,2048,0.01,294,1,2,"[[25292343.0, 5.105129], [25129117.0, 5.109422...","[[339.9169921875, -0.2797656601000824], [6.921..."
2,smica,2.01,False,1.2,38,2048,0.002,242,0,0,"[[25276177.0, 5.470043], [25120330.0, 5.470631...","[[349.4970703125, -0.24246333228779804], [340...."
3,smica,2.01,False,1.2,38,2048,0,0,0,0,[],[]
4,smica,2.01,False,1.2,39,2048,0.05,728,22,54,"[[24016663.0, 4.629977], [25776567.0, 4.630096...","[[79.78271484375, 2.612040471496698], [19.2919..."
5,smica,2.01,False,1.2,39,2048,0.01,558,2,10,"[[41960069.0, 5.042193], [25103369.0, 5.042642...","[[208.74633431085044, -41.86035581487161], [31..."
6,smica,2.01,False,1.2,39,2048,0.002,482,0,2,"[[24344654.0, 5.388754], [25498200.0, 5.392681...","[[93.44970703125, 1.865426534502518], [26.3671..."
7,smica,2.01,False,1.2,39,2048,0,0,0,0,[],[]
8,smica,2.01,False,1.2,40,2048,0.05,1565,39,180,"[[24123141.0, 4.542908], [10469814.0, 4.543569...","[[78.9697265625, 2.3693484086689125], [199.270..."
9,smica,2.01,False,1.2,40,2048,0.01,1197,4,63,"[[25554534.0, 4.956656], [24285589.0, 4.959849...","[[341.982421875, -0.8766297866493602], [17.797..."


Map number 29
Map number 33
Map number 37


...

Map number 101
Map number 105
Map number 109
Map number 113
Map number 117
Map number 121


,Map,version,inpainted,B,j,nside,alpha,detections,outside common mask,outside inpainting mask,canlist,angs
0,smica,2.01,False,1.2,38,2048,0.05,365,14,25,"[[17969506.0, 4.725307], [42997244.0, 4.734639...","[[15.57861328125, 16.60711032435087], [182.185..."
1,smica,2.01,False,1.2,38,2048,0.01,294,1,2,"[[25292343.0, 5.105129], [25129117.0, 5.109422...","[[339.9169921875, -0.2797656601000824], [6.921..."
2,smica,2.01,False,1.2,38,2048,0.002,242,0,0,"[[25276177.0, 5.470043], [25120330.0, 5.470631...","[[349.4970703125, -0.24246333228779804], [340...."
3,smica,2.01,False,1.2,38,2048,0,0,0,0,[],[]
4,smica,2.01,False,1.2,39,2048,0.05,728,22,54,"[[24016663.0, 4.629977], [25776567.0, 4.630096...","[[79.78271484375, 2.612040471496698], [19.2919..."
5,smica,2.01,False,1.2,39,2048,0.01,558,2,10,"[[41960069.0, 5.042193], [25103369.0, 5.042642...","[[208.74633431085044, -41.86035581487161], [31..."
6,smica,2.01,False,1.2,39,2048,0.002,482,0,2,"[[24344654.0, 5.388754], [25498200.0, 5.392681...","[[93.44970703125, 1.865426534502518], [26.3671..."
7,smica,2.01,False,1.2,39,2048,0,0,0,0,[],[]
8,smica,2.01,False,1.2,40,2048,0.05,1565,39,180,"[[24123141.0, 4.542908], [10469814.0, 4.543569...","[[78.9697265625, 2.3693484086689125], [199.270..."
9,smica,2.01,False,1.2,40,2048,0.01,1197,4,63,"[[25554534.0, 4.956656], [24285589.0, 4.959849...","[[341.982421875, -0.8766297866493602], [17.797..."


Map number 125
Map number 129
Map number 133
Map number 137
Map number 141
Map number 145


,Map,version,inpainted,B,j,nside,alpha,detections,outside common mask,outside inpainting mask,canlist,angs
0,smica,2.01,False,1.2,38,2048,0.05,365,14,25,"[[17969506.0, 4.725307], [42997244.0, 4.734639...","[[15.57861328125, 16.60711032435087], [182.185..."
1,smica,2.01,False,1.2,38,2048,0.01,294,1,2,"[[25292343.0, 5.105129], [25129117.0, 5.109422...","[[339.9169921875, -0.2797656601000824], [6.921..."
2,smica,2.01,False,1.2,38,2048,0.002,242,0,0,"[[25276177.0, 5.470043], [25120330.0, 5.470631...","[[349.4970703125, -0.24246333228779804], [340...."
3,smica,2.01,False,1.2,38,2048,0,0,0,0,[],[]
4,smica,2.01,False,1.2,39,2048,0.05,728,22,54,"[[24016663.0, 4.629977], [25776567.0, 4.630096...","[[79.78271484375, 2.612040471496698], [19.2919..."
5,smica,2.01,False,1.2,39,2048,0.01,558,2,10,"[[41960069.0, 5.042193], [25103369.0, 5.042642...","[[208.74633431085044, -41.86035581487161], [31..."
6,smica,2.01,False,1.2,39,2048,0.002,482,0,2,"[[24344654.0, 5.388754], [25498200.0, 5.392681...","[[93.44970703125, 1.865426534502518], [26.3671..."
7,smica,2.01,False,1.2,39,2048,0,0,0,0,[],[]
8,smica,2.01,False,1.2,40,2048,0.05,1565,39,180,"[[24123141.0, 4.542908], [10469814.0, 4.543569...","[[78.9697265625, 2.3693484086689125], [199.270..."
9,smica,2.01,False,1.2,40,2048,0.01,1197,4,63,"[[25554534.0, 4.956656], [24285589.0, 4.959849...","[[341.982421875, -0.8766297866493602], [17.797..."


'00:08:35'

In [8]:
pandas.set_option('display.max_rows',600)
pandas.set_option('display.max_columns',30)

In [8]:
results.T

,Map,version,inpainted,B,j,nside,alpha,detections,outside common mask,outside inpainting mask,canlist,angs
0,smica,2.01,False,1.2,38,2048,0.05,365,14,25,"[[17969506.0, 4.725307], [42997244.0, 4.734639...","[[15.57861328125, 16.60711032435087], [182.185..."
1,smica,2.01,False,1.2,38,2048,0.01,294,1,2,"[[25292343.0, 5.105129], [25129117.0, 5.109422...","[[339.9169921875, -0.2797656601000824], [6.921..."
2,smica,2.01,False,1.2,38,2048,0.002,242,0,0,"[[25276177.0, 5.470043], [25120330.0, 5.470631...","[[349.4970703125, -0.24246333228779804], [340...."
3,smica,2.01,False,1.2,38,2048,0,0,0,0,[],[]
4,smica,2.01,False,1.2,39,2048,0.05,728,22,54,"[[24016663.0, 4.629977], [25776567.0, 4.630096...","[[79.78271484375, 2.612040471496698], [19.2919..."
5,smica,2.01,False,1.2,39,2048,0.01,558,2,10,"[[41960069.0, 5.042193], [25103369.0, 5.042642...","[[208.74633431085044, -41.86035581487161], [31..."
6,smica,2.01,False,1.2,39,2048,0.002,482,0,2,"[[24344654.0, 5.388754], [25498200.0, 5.392681...","[[93.44970703125, 1.865426534502518], [26.3671..."
7,smica,2.01,False,1.2,39,2048,0,0,0,0,[],[]
8,smica,2.01,False,1.2,40,2048,0.05,1565,39,180,"[[24123141.0, 4.542908], [10469814.0, 4.543569...","[[78.9697265625, 2.3693484086689125], [199.270..."
9,smica,2.01,False,1.2,40,2048,0.01,1197,4,63,"[[25554534.0, 4.956656], [24285589.0, 4.959849...","[[341.982421875, -0.8766297866493602], [17.797..."


In [126]:
# totcata=Catalogs(catalogsfolder)
# cata30=Catalogs(catalogsfolder,key='030')
goodcata=Catalogs(catalogsfolder,key='COM',notkey='excluded')
exccata=Catalogs(catalogsfolder,key='excluded')
pscata=Catalogs(catalogsfolder,key='COM')
# szcata=Catalogs(catalogsfolder,key='SZ')
# cccata=Catalogs(catalogsfolder,key='GCC')
# hzcata=Catalogs(catalogsfolder,key='HZ')

In [208]:
time1=time.time()
catresults=pandas.DataFrame()
index2=['Cat PS', 'Cat PS_good', 'Cat PS_excl', 'Cat PS mask', 'Cat PS_good mask', 'Cat PS_excl mask']
my_shelve = shelve.open(shelvefile,'n')      #'n' for new
my_shelve['catresults'] = catresults
my_shelve['results']=results
for i in results.columns:
    print(i,time.strftime("%H:%M:%S", time.gmtime(time.time()-time1)))
    angs=results[i]['angs']
    canlist=results[i]['canlist']
    if len(canlist)>0:
#         tot=totcata.check(angs,tol=angtol)
        good=goodcata.check(angs,tol=angtol)
        excl=exccata.check(angs,tol=angtol)
        ps=pscata.check(angs,tol=angtol)
#         sz=szcata.check(angs,tol=angtol)
#         cc=cccata.check(angs,tol=angtol)
#         hz=hzcata.check(angs,tol=angtol)
        data2=np.array([ps,good,excl])
        angs=angs[np.array(1-np.isin(canlist[:,0],np.where(maskcom==0)),dtype=bool)]
#         tot=totcata.check(angs,tol=angtol)
        good=goodcata.check(angs,tol=angtol)
        excl=exccata.check(angs,tol=angtol)
        ps=pscata.check(angs,tol=angtol)
#         sz=szcata.check(angs,tol=angtol)
#         cc=cccata.check(angs,tol=angtol)
#         hz=hzcata.check(angs,tol=angtol)
        data2=np.append(data2,np.array([ps,good,excl]))
        print(data2)
        catresults[i]=pandas.Series(data2,index=index2)
    else:
        catresults[i]=pandas.Series(np.zeros(6).astype('int'),index=index2)
    my_shelve['catresults'] = catresults
my_shelve['results']=results
my_shelve.close()

time2=time.time()
time.strftime("%H:%M:%S", time.gmtime(time2-time1))

2 00:00:00
[36  4 36  0  0  0]
6 00:00:10
[58  5 56  0  0  0]
10 00:00:29
[114  14 107   0   0   0]
14 00:01:09
[337 190 252   0   0   0]
18 00:01:22
[658 396 454   0   0   0]
22 00:01:50
[1126  639  774    3    2    2]
26 00:02:47
[23 23 23  0  0  0]
30 00:02:49
[49 48 47  0  0  0]
34 00:02:51
[83 78 72  0  0  0]
38 00:02:53
[1840   61 1828    0    0    0]
42 00:04:50
[3669   76 3655    4    3    2]
46 00:08:13
[6594  104 6568    7    5    3]
50 00:14:11
[1470   55 1461    0    0    0]
54 00:15:23
[3110   68 3099    2    1    2]
58 00:22:34
[6001   95 5982    6    4    3]
62 00:30:08
66 00:30:08
[0 0 0 0 0 0]
70 00:30:09
[0 0 0 0 0 0]
74 00:30:10
[107  71  95   0   0   0]
78 00:30:13
[171  97 152   0   0   0]
82 00:30:21
[317 125 290   1   0   1]
86 00:30:38
[1 1 1 0 0 0]
90 00:30:39
[5 4 4 0 0 0]
94 00:30:40
[6 5 5 0 0 0]
98 00:30:41
[340 193 255   0   0   0]
102 00:30:54
[660 400 453   0   0   0]
106 00:31:21
[1089  631  740    2    2    1]
110 00:32:17
114 00:32:17
118 00:32:18
122

'00:41:53'

In [210]:
ncatresults

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143
Cat PS,52,42,36,0,97,69,58,0,214,153,114,0,491,409,337,...,5209,4737,552,9,8,7,0,12,8,7,0,20,13,10,0
Cat PS_good,8,4,4,0,20,9,5,0,49,22,14,0,299,233,190,...,415,361,146,4,3,3,0,5,4,3,0,6,4,4,0
Cat PS_excl,48,42,36,0,82,64,56,0,179,141,107,0,338,302,252,...,5116,4676,543,8,7,6,0,9,6,5,0,17,11,8,0
Cat PS mask,1,0,0,0,5,0,0,0,9,2,0,0,6,0,0,...,11,3,0,0,0,0,0,0,0,0,0,0,0,0,0
Cat PS_good mask,1,0,0,0,4,0,0,0,8,1,0,0,6,0,0,...,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Cat PS_excl mask,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,...,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0


In [214]:
totresults=pandas.concat([results,ncatresults]).T

In [218]:
my_shelve = shelve.open(shelvefile,'n')      #'n' for new

my_shelve['results'] = results
my_shelve['catresults'] = catresults
my_shelve['totresults'] = totresults

my_shelve.close()

In [9]:
index_to_show=['Map', 'version', 'inpainted', 'j', 'alpha', 'detections','Cat PS',
        'outside common mask', 'outside inpainting mask']
order=['alpha','version','Map','inpainted','j']
query='detections>0 & alpha==0 '
totresults.sort_values(order).query(query)[index_to_show]

,Map,version,inpainted,j,alpha,detections,Cat PS,outside common mask,outside inpainting mask
39,sevem,2.01,False,38,0,99,79,0,0
43,sevem,2.01,False,39,0,192,165,0,0
47,sevem,2.01,False,40,0,397,349,0,0
123,commander,3.00,False,38,0,713,272,0,76
127,commander,3.00,False,39,0,907,404,0,55
131,commander,3.00,False,40,0,1026,552,0,12
51,sevem,3.00,False,38,0,71,62,0,0
55,sevem,3.00,False,39,0,160,138,0,0
59,sevem,3.00,False,40,0,354,302,0,0
75,smica,3.00,False,38,0,1,1,0,0


In [10]:
index_to_show=['Map', 'version', 'inpainted','B', 'j', 'nside', 'alpha', 'detections',
        'outside common mask', 'outside inpainting mask','Cat PS', 'Cat PS_good', 
        'Cat PS_excl', 'Cat PS mask', 'Cat PS_good mask', 'Cat PS_excl mask']
order=['Map','version','inpainted','j','alpha']
query='j==39 & alpha==0.01'
totresults.sort_values(order).query(query)[index_to_show]

,Map,version,inpainted,B,j,nside,alpha,detections,outside common mask,outside inpainting mask,Cat PS,Cat PS_good,Cat PS_excl,Cat PS mask,Cat PS_good mask,Cat PS_excl mask
29,commander,2.01,False,1.2,39,2048,0.01,55,0,2,53,52,50,0,0,0
125,commander,3.00,False,1.2,39,2048,0.01,6335,37,717,2927,329,2873,11,8,3
137,commander,3.00,True,1.2,39,2048,0.01,70,1,12,8,4,6,0,0,0
17,nilc,2.01,False,1.2,39,2048,0.01,1222,6,10,753,449,519,1,1,1
101,nilc,3.00,False,1.2,39,2048,0.01,1240,8,15,755,452,520,1,1,1
113,nilc,3.00,True,1.2,39,2048,0.01,0,0,0,0,0,0,0,0,0
41,sevem,2.01,False,1.2,39,2048,0.01,7064,39,690,4056,90,4029,13,10,4
53,sevem,3.00,False,1.2,39,2048,0.01,5894,31,261,3424,77,3406,10,7,4
65,sevem,3.00,True,1.2,39,2048,0.01,3,0,1,0,0,0,0,0,0
5,smica,2.01,False,1.2,39,2048,0.01,558,2,10,69,9,64,0,0,0
